# **Quadruped Robot Trajectory Motion**
## Nischay N
## Pes University, Bangalore

In [65]:
import numpy as np
from numpy import pi,sin,cos

import PyQt5
import math
import time
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from IPython import get_ipython
get_ipython().run_line_magic('matplotlib','qt')

Defining the Values

In [66]:
# Trajectory Plan
clr_height = 4    # cm
L_span = 8      # cm
v_d = 0.7         # cm/s

# Stance Phase Period
T_st = 2*L_span/(100*v_d)
# Swing Phase Period(s) 
T_sw = 0.15               

T_stride = T_st + T_sw
precision = 0.01

legNum = 1

# Stance Phase
S_st_i = 5
# Swing Phase
S_sw_i = 5

phase = np.array([0.5,0.5,0.5,0.5])

loopstart = 0
loopEnd = 0
looptime = 0

# Control Points
ds = [0,0.5,0,0.5]

# Empty Variables for Each Leg
t1 = 0
t2 = 0
t3 = 0
t4 = 0

t_elapse_ref = 0.2    # 0 to 0.33 Variation

# Leg Time for Each Leg
t_i = np.array([t1,t2,t3,t4])

# Defining Empty Variables to Store x, y and z co-ordinates
x = 0
y = 0
z = 0

bodyHeight = 18 #cm
delta = -0.85

# Berstein Control Points
beziercontrolpoints = np.array([[-L_span, 0.0-bodyHeight],
                       [-L_span*1.4, 0.0-bodyHeight],
                       [-L_span*1.5, clr_height*0.9-bodyHeight],
                       [-L_span*1.5, clr_height*0.9-bodyHeight],
                       [-L_span*1.5, clr_height*0.9-bodyHeight],
                       [0.0, clr_height*0.9-bodyHeight],
                       [0.0, clr_height*0.9-bodyHeight],
                       [0.0, clr_height*1.157-bodyHeight],
                       [-L_span*1.5, clr_height*1.157-bodyHeight],
                       [-L_span*1.5, clr_height*1.157-bodyHeight],
                       [-L_span*1.4, 0.0-bodyHeight],
                       [-L_span, 0.0-bodyHeight]])

# femur - upper leg
femur = 12

#tibia - lower leg
tibia = 11.5

# Hip
hip = 0

# Define All Empty Arrays to Store Angles, X and Z Values for Each Leg
angles       = []
alpha_list   = []
beta_list    = []
gamma_list   = []
alpha_list2  = []
beta_list2   = []
gamma_list2  = []
alpha_list3  = []
beta_list3   = []
gamma_list3  = []
alpha_list4  = []
beta_list4   = []
gamma_list4  = []

x_list1 = []
z_list1 = []
x_list2 = []
z_list2 = []
x_list3 = []
z_list3 = []
x_list4 = []
z_list4 = []

Defining the Required Function

In [67]:
def nCr(n,r):
  f = math.factorial
  return f(n) // f(r) // f(n-r)

def berstein(S,N,K,v):
  return nCr(N,K)*((1-S)**(N-K))*(S**K)*(v)

# Inverse Kinematics 
def legIK(x,y,z):
  R = math.sqrt(x**2 + y**2 + z**2)
  R1 = math.sqrt(z**2 + y**2)

  theta1 = math.atan2(-z,y)
  theta2 = math.acos(hip/R1)
  alpha = (theta2 - theta1)*180/math.pi

  R2 = math.sqrt(R**2 - hip**2)
  phi1 = math.asin(x/R2)
  temp1 = (femur**2 + R2**2 - tibia**2)/(2*femur*R2)

  if temp1 > 1:
    temp1 = 1
  if temp1 < -1:
    temp1 = -1

  phi2 = math.acos(temp1)
  beta = (phi1 - phi2)*180/math.pi

  temp2 = (femur**2 - R2**2 + tibia**2)/(2*femur*tibia)
  if temp2 >1:
    temp2 = 1
  if temp2 < -1:
    temp2 = -1
  
  psi = math.acos(temp2)
  gamma = (math.pi - psi)*180/math.pi

  return np.array([alpha,beta,gamma])

Calculation of Robot Motion

In [68]:
t = 0.0   # Clock Started
t_TD_ref = t    # Initialized from Touch Down
start = time.perf_counter()     # Random time in sec
legNum = 1
loopEnd = 0
loopStart = 0

# No.of Cycles
for i in range(0, 1):
  S_st_i = 0
  S_sw_i = 0
  TD = False  # Touch Down Value

  while (not TD):
    loopstart = time.perf_counter()
    t_elapse_ref = t-t_TD_ref
    TD = False

    if t_elapse_ref >= T_stride:
      t_elapse_ref = 0
      TD = True

    if TD:
      t_TD_ref = t

    t_i = t_elapse_ref - T_stride*np.array(ds)


    for legTime in t_i:
      # Current Stance Phase
      if legTime>0 and legTime<T_st:
        # calculate x, y, z co-ordinates
        x = 0
        y = 0
        z = 0

        S_st_i = legTime/T_st
        phase[legNum-1] = S_st_i
        x = L_span*(1- 2*S_st_i)+0
        z = delta*(math.cos(math.pi*x/(2*L_span))+0)- bodyHeight

        # Angles of Legs
        angles = legIK(x,y,z)

      # Old Swing Phase
      if legTime>= -T_sw and legTime<=0:
        x = 0
        y = 0
        z = 0

        S_sw_i = (legTime+T_sw)/T_sw
        phase[legNum-1] = S_sw_i
        
        for index in range(0,12):
          x+=berstein(S_sw_i,11,index,beziercontrolpoints[index][0])
          z+=berstein(S_sw_i,11,index,beziercontrolpoints[index][1])

        # Angles of the Legs
        angles = legIK(x,y,z)
        

      # Current Swing Phase
      elif legTime>= T_st and legTime<=T_stride:
        x = 0
        y = 0
        z = 0

        S_sw_i = (legTime-T_sw)/T_sw
        phase[legNum-1] = S_sw_i

        for index in range(0,12):
          x+=berstein(S_sw_i,11,index,beziercontrolpoints[index][0])
          z+=berstein(S_sw_i,11,index,beziercontrolpoints[index][1])

        # Angles of the Legs
        angles = legIK(x,y,z)
      

      if legNum == 1:
        # Storing Each Angle Values
        val0 = angles[0]*math.pi/180
        val1 = angles[1]*math.pi/180
        val2 = angles[2]*math.pi/180

        # Storing All Angle Values in Variables
        alpha_list.append(val0)
        beta_list.append(val1)
        gamma_list.append(val2)

        x_list1.append(x)
        z_list1.append(z)

      if legNum == 2:
        # Storing Each Angle Values
        val0 = angles[0]*math.pi/180
        val1 = angles[1]*math.pi/180
        val2 = angles[2]*math.pi/180

        # Storing All Angle Values in Variables
        alpha_list2.append(val0)
        beta_list2.append(val1)
        gamma_list2.append(val2)

        x_list2.append(x)
        z_list2.append(z)

      if legNum == 3:
        # Storing Each Angle Values
        val0 = angles[0]*math.pi/180
        val1 = angles[1]*math.pi/180
        val2 = angles[2]*math.pi/180

        # Storing All Angle Values in Variables
        alpha_list3.append(val0)
        beta_list3.append(val1)
        gamma_list3.append(val2)

        x_list3.append(float(x)+40)
        z_list3.append(z)

      if legNum == 4:
        # Storing Each Angle Values
        val0 = angles[0]*math.pi/180
        val1 = angles[1]*math.pi/180
        val2 = angles[2]*math.pi/180
        # Storing All Angle Values in Variables
        alpha_list4.append(val0)
        beta_list4.append(val1)
        gamma_list4.append(val2)

        x_list4.append(float(x)+40)
        z_list4.append(z)
        
      legNum+=1
    legNum = 1
    t_elapse_ref += precision
    t += precision
    loopEnd = time.perf_counter()

    loopTime = loopEnd - loopStart


Defining Empty Arrays to Store Leg Position

In [69]:
# Leg Parameters
Femur_one_leg = femur
Tibia_one_leg = tibia

# Define Empty Variables to Store x and y co-ordinate for each leg

## Leg 1
# Femur
X1 = np.zeros(len(beta_list))
Y1 = np.zeros(len(beta_list))
# Tibia
X2 = np.zeros(len(gamma_list))
Y2 = np.zeros(len(gamma_list))

## Leg 2
# Femur
X1_2 = np.zeros(len(beta_list2))
Y1_2 = np.zeros(len(beta_list2))
# Tibia
X2_2 = np.zeros(len(gamma_list2))
Y2_2 = np.zeros(len(gamma_list2))

## Leg 3
# Femur
X1_3 = np.zeros(len(beta_list3))
Y1_3 = np.zeros(len(beta_list3))
# Tibia
X2_3 = np.zeros(len(gamma_list3))
Y2_3 = np.zeros(len(gamma_list3))

## Leg 4
# Femur
X1_4 = np.zeros(len(beta_list4))
Y1_4 = np.zeros(len(beta_list4))
# Tibia
X2_4 = np.zeros(len(gamma_list4))
Y2_4 = np.zeros(len(gamma_list4))

Calculate the Quadruped Robot Leg Position

In [70]:
## Leg 1
for index, (beta,gamma) in enumerate(zip(beta_list,gamma_list)):
  # Femur
  x1 = Femur_one_leg*cos(-beta - (pi/2))
  y1 = Femur_one_leg*sin(-beta - (pi/2))
  # Tibia
  x2 = x1+Tibia_one_leg*cos(-pi/2 - (beta+gamma))
  y2 = y1+Tibia_one_leg*sin(-pi/2 - (beta+gamma))

  # Storing position values for each beta and gamma angles
  X1[index] = x1
  Y1[index] = y1
  X2[index] = x2
  Y2[index] = y2

## Leg 2
for index2, (beta2,gamma2) in enumerate(zip(beta_list2,gamma_list2)):
  # Femur
  x1_2 = Femur_one_leg*cos(-beta2 - (pi/2))
  y1_2 = Femur_one_leg*sin(-beta2 - (pi/2))
  # Tibia
  x2_2 = x1_2+Tibia_one_leg*cos(-pi/2 - (beta2+gamma2))
  y2_2 = y1_2+Tibia_one_leg*sin(-pi/2 - (beta2+gamma2))

  # Storing position values for each beta and gamma angles
  X1_2[index2] = x1_2
  Y1_2[index2] = y1_2
  X2_2[index2] = x2_2
  Y2_2[index2] = y2_2

## Leg 3
for index3, (beta3,gamma3) in enumerate(zip(beta_list3,gamma_list3)):
  # Femur
  x1_3 = 40+ Femur_one_leg*cos(-beta3 - (pi/2))
  y1_3 = Femur_one_leg*sin(-beta3 - (pi/2))
  # Tibia
  x2_3 = x1_3+Tibia_one_leg*cos(-pi/2 - (beta3+gamma3))
  y2_3 = y1_3+Tibia_one_leg*sin(-pi/2 - (beta3+gamma3))

  # Storing position values for each beta and gamma angles
  X1_3[index3] = x1_3
  Y1_3[index3] = y1_3
  X2_3[index3] = x2_3
  Y2_3[index3] = y2_3

## Leg 4
for index4, (beta4,gamma4) in enumerate(zip(beta_list4,gamma_list4)):
  # Femur
  x1_4 = 40+Femur_one_leg*cos(-beta4 - (pi/2))
  y1_4 = Femur_one_leg*sin(-beta4 - (pi/2))
  # Tibia
  x2_4 = x1_4+Tibia_one_leg*cos(-pi/2 - (beta4+gamma4))
  y2_4 = y1_4+Tibia_one_leg*sin(-pi/2 - (beta4+gamma4))

  # Storing position values for each beta and gamma angles
  X1_4[index4] = x1_4
  Y1_4[index4] = y1_4
  X2_4[index4] = x2_4
  Y2_4[index4] = y2_4



Visualization

In [71]:
# Set the Parameters of the Animation
fig = plt.figure()
ax = fig.add_subplot(111, aspect = 'equal', autoscale_on = False, xlim = (-30,70), ylim = (-50, 50))
ax.grid()
ax.set_title('Quadruped Robot Motion')

# Setting Line Color For Each Line
line,  = ax.plot([],[],'o-',lw = 5, color='Brown')
line2, = ax.plot([],[],'o-',lw = 5, color='Blue')
line3, = ax.plot([],[],'o-',lw = 5, color='Brown')
line4, = ax.plot([],[],'o-',lw = 5, color='Blue')

# Initialization Function
def init():
  line.set_data([],[])
  line2.set_data([],[])
  line3.set_data([],[])
  line4.set_data([],[])

  return line,line2,line3,line4

# Animation Function
def animate(i):
  x_points = [0, X1[i], X2[i]]
  y_points = [0, Y1[i], Y2[i]]

  x2_points = [0, X1_2[i], X2_2[i]]
  y2_points = [0, Y1_2[i], Y2_2[i]]

  x3_points = [40, X1_3[i], X2_3[i]]
  y3_points = [0, Y1_3[i], Y2_3[i]]

  x4_points = [40, X1_4[i], X2_4[i]]
  y4_points = [0, Y1_4[i], Y2_4[i]]

  line.set_data(x_points, y_points)
  line2.set_data(x2_points, y2_points)
  line3.set_data(x3_points, y3_points)
  line4.set_data(x4_points, y4_points)

  return line,line2,line3,line4


# Creating the Animation of the Robot
ani = animation.FuncAnimation(fig, animate, init_func= init, frames= len(X1), interval = 100, blit = True, repeat = True)

# Plotting the Reference Circle for the Robot Legs to Follow
plt.plot(X2,Y2, '#05143b')
plt.plot(X2_2,Y2_2, '#37acf0')
plt.plot(X2_3,Y2_3, '#05143b')
plt.plot(X2_4,Y2_4, '#05143b')

# PLotting the Legs 
# Ground Plot
plt.plot([-20,60],[-bodyHeight,-bodyHeight], 'brown')

# Body Plot
plt.plot([-4,44],[0,0],'#010b24')
plt.plot([-4,-4],[0,5],'#010b24')
plt.plot([44,44],[0,5],'#010b24')
plt.plot([-4,44],[5,5],'#010b24')

for ind in range(100):
  plt.plot([-4,44],[ind*5/100,ind*5/100],'black')